<div style="background-color: #ccffcc; padding: 10px;">
    <h1> Tutorial 2 </h1>
    <h2> Introduction to Modal Decomposition </h2>
</div>

# Overview

This Jupyter notebook demonstrates how modal decomposition methods can be used for flow feature extraction in fluid mechanics datasets. Modal decomposition techniques, such as Proper Orthogonal Decomposition (POD) and Dynamic Mode Decomposition (DMD), help identify coherent structures in fluid flows, providing a useful dimension reduction for subsequent reduced order modelling. The example application focuses on the classic problem of fluid flow past a cylinder, showing how these methods can simplify complex flow fields into a manageable number of modes. This dimension reduction enables efficient and accurate reduced order modelling using Sparse Identification of Nonlinear Dynamics (SINDy).

## Recommended reading

* [Modal Analysis of Fluid Flows: An Overview](https://doi.org/10.2514/1.J056060)
* [Reduced order modelling](https://uk.mathworks.com/discovery/reduced-order-modeling.html)
* [Dynamic Mode Decomposition (DMD) of numerical and experimental data](https://doi.org/10.1017/S0022112010001217)
* [Proper Orthogonal Decomposition (POD) MIT notes](http://web.mit.edu/6.242/www/images/lec6_6242_2004.pdf)

<hr>

<div style="background-color: #e6ccff; padding: 10px;">

<h1> Machine Learning Theory </h1>

# Modal Decomposition

## The problem

Flow feature extraction in fluid mechanics datasets involves identifying and characterizing significant patterns and structures within fluid flow data. This process is helpful for understanding complex flow behaviors, such as turbulence, vortex dynamics, and boundary layer interactions. By extracting these features, researchers can gain insights into the underlying physics of fluid flows and improve predictive models.

Modal decomposition methods, such as Proper Orthogonal Decomposition (POD) and Dynamic Mode Decomposition (DMD), are powerful tools for flow feature extraction. These methods decompose complex flow fields into a set of orthogonal modes, each representing a distinct flow feature. By analyzing these modes, researchers can isolate and study specific flow phenomena, leading to a deeper understanding of fluid dynamics and more efficient data analysis.

## Popular modal decomposition methods

* Singular Value Decomposition (SVD): A fundamental linear algebra technique used to decompose a matrix into its singular values and vectors, often used in various modal analysis methods.
* Proper Orthogonal Decomposition (POD): Also known as Principal Component Analysis (PCA) in statistics, POD identifies the most energetic modes in a flow field.
* Dynamic Mode Decomposition (DMD): A method that decomposes complex systems into modes with specific temporal behaviors, useful for analyzing dynamic features in fluid flows.
* Fourier Decomposition: Decomposes a signal into its constituent frequencies, often used for periodic or quasi-periodic flows.
* Wavelet Decomposition: Provides a time-frequency representation of a signal, useful for analyzing transient and multi-scale phenomena in fluid flows.

<div style="background-color: #cce5ff; padding: 10px;">

# Python

## [SciPy](https://scipy.org/)

SciPy is a widely used open-source library for scientific and technical computing in Python. It builds on the NumPy array object and provides a large collection of algorithms and functions for numerical integration, optimization, signal processing, linear algebra, and more. SciPy enables users to perform complex scientific computations with ease and efficiency. With its intuitive Python interface, SciPy is accessible for beginners, yet it also offers advanced capabilities for experienced programmers. SciPy is compatible with various platforms, from personal computers to high-performance computing environments.

## [PySINDy](https://github.com/dynamicslab/pysindy)

PySINDy is a sparse regression package with several implementations for the Sparse Identification of Nonlinear Dynamical systems (SINDy) method introduced in Brunton et al. (2016a), including the unified optimization approach of Champion et al. (2019), SINDy with control from Brunton et al. (2016b), Trapping SINDy from Kaptanoglu et al. (2021), SINDy-PI from Kaheman et al. (2020), PDE-FIND from Rudy et al. (2017), and so on. A comprehensive literature review is given in de Silva et al. (2020) and Kaptanoglu, de Silva et al. (2021).

## Further reading

If you want to run this notebook locally or on a remote service:

* [running Jupyter notebooks](https://jupyter.readthedocs.io/en/latest/running.html)
* [installing the required Python environments](https://github.com/cemac/LIFD_ENV_ML_NOTEBOOKS/blob/main/howtorun.md)
* [running the Jupyter notebooks locally](https://github.com/cemac/LIFD_ENV_ML_NOTEBOOKS/blob/main/jupyter_notebooks.md)

</div>

<hr>

<div style="background-color: #ffffcc; padding: 10px;">
    
<h1> Requirements </h1>

This notebook should run with the following requirements satisfied.

<h2> Python Packages: </h2>

* numpy
* scipy
* matplotlib
* notebook
* pysindy
* scikit-learn

<h2> Data Requirements</h2>

Required data from the fluid dynamics simulations are already included in the repository as `.npz` files.

</div>

**Contents:**

1. [Overview and machine-learning theory](#Overview)
2. [Singular Value Decomposition (SVD)](#Part-1:-SVD)
3. [Proper Orthogonal Decomposition (POD)](#Part-2:-POD)
4. [Dynamic Mode Decomposition (DMD)](#Part-3:-DMD)

<div style="background-color: #cce5ff; padding: 10px;">

## Import modules

First we will import all the modules needed during this tutorial.

</div>

### Note for Colab users

If you are using Google Colab to run this notebook, you will need to download an additional module now by uncommenting and running the next code cell.

In [ ]:
# !wget https://raw.githubusercontent.com/cemac/LIFD_ModalDecomposition/refs/heads/main/helper_functions.py

Let's import all the libraries we need. This may take a few seconds, depending on the speed of your filesystem.

In [ ]:
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
from sklearn.datasets import load_sample_image
from helper_functions import plot_cylinder_data
from helper_functions import download_data
import matplotlib.animation
from IPython.display import HTML

## Part 1: SVD

Let's start by reviewing the Singular Value Decomposition (SVD). The SVD is a powerful linear algebra technique that decomposes a matrix $\mathbf{A}$ as $\mathbf{A}=\mathbf{U}\boldsymbol{\Sigma}\mathbf{V}^T$, where $T$ denotes the matrix transpose. Note that we have restricted ourselves to real matrices for this notebook, but everything carries over to the case of complex matrices where transposes become Hermitian transposes. The column vectors of $\mathbf{U}$ and $\mathbf{V}$ are known as the left and right singular vectors, respectively. Furthermore, the matrix $\boldsymbol{\Sigma}$ is diagonal with positive real entries. Denoting these column vectors as $\mathbf{u}_j$ and $\mathbf{v}_j$, we can also write the singular triplet $(\mathbf{u}_j, \mathbf{v}_j, \sigma_j)$, where $\sigma_j$ is $\boldsymbol{\Sigma}_{jj}$. The matrices $\mathbf{U}$ and $\mathbf{V}$ are unitary, meaning that $\{\mathbf{u}_j\}_{j=0}$ and $\{\mathbf{v}_j\}_{j=0}$ form orthogonal bases.

By definition we then have that $\mathbf{A}\mathbf{v}_j=\sigma_j\mathbf{u}_j$, showing that the action of $\mathbf{A}$ on any vector can be approximated well by the sum of a handful of vectors $\mathbf{u}_j$ provided the singular values decay quickly. In other words, if we have the ordering $\sigma_0>\sigma_1>...$, then if $\sigma_0\gg\sigma_1\gg...$ the SVD can be used to create a low-rank approximation to $\mathbf{A}$.

To illustrate this, let's consider compressing an image.

We first load an image of a flower, and rescale the integer red green blue (RGB) data to be floats between zero and one.

In [ ]:
# Load a flower image, and rescale the RGB channels to lie within [0, 1]
flower = np.float32(load_sample_image('flower.jpg')/255)
channels = ['red', 'green', 'blue']

In [ ]:
def plot_image(image: np.ndarray, ax=None, title=None):
    """
    Plots an image from RGB data.
    
    Parameters
    ----------  
    image: array with shape (number of pixels in y-direction,
                             number of pixels in x-direction,
                             channels).
    ax: axis to plot the image in.
    title: title for the plot.
    """
    if ax is None:
        fig, ax = plt.subplots(1)
    ax.imshow(image)
    ax.axis('off')
    ax.set_title(title)

In [ ]:
plot_image(flower, title='Original image')

In [ ]:
print('Shape of our image:', flower.shape)
print('Memory of flower: %.2f MB ' % (np.prod(flower.shape)*32/1024/1024))

This image takes $427\times640\times3\times\textrm{size of  float}\approx 25 ~\mathrm{MB}$. Let's see if we can compress the image by retaining a low-rank approximation where only some of the singular values are kept.

In [ ]:
channels = ['red', 'green', 'blue']

In [ ]:
def image_reduced(image, rank=None):
    """
    Return a low-rank approximation for an image.

    Parameters
    ----------
    image: array with shape (number of pixels in y-direction,
                             number of pixels in x-direction,
                             number of channels).
    rank: How many dominant singular values to keep (default: all of them).

    Returns
    -------
    reduced_image: A dictionary with size, red, green, and blue keys. The
                   entry of size contains the shape of the image.
                   Each channel (red, green, blue) entry is a tuple 
                   (U, Sigma, VH), such that our low rank approximation 
                   for that channel is U@Sigma@VH.
    """
    reduced_image = {'size': image.shape}
    if rank == None:
        rank = np.min(image.shape[:2]) - 1
    # Loop over RGB channels
    for i, channel in enumerate(channels):
        U, S, VH = sp.linalg.svds(flower[:, :, i], k=rank)
        reduced_image[channel] = (U, S, VH)
    return reduced_image

In [ ]:
def reconstruct_image(low_rank_image):
    """
    Reconstructs the image from the low rank approximation.

    Parameters
    ----------
    low_rank_image: Low rank approximation obtained from image_reduced.
    """
    # Reconstruct image
    reconstructed_image = np.empty(shape=low_rank_image['size'], dtype=np.float32)
    for i, channel in enumerate(channels):
        (U, S, VH) = low_rank_image[channel]
        reconstructed_image[:, :, i] = U @ np.diag(S) @ VH
    return reconstructed_image

In [ ]:
ranks = [1, 10, 40, 100]
reduced_images = []
for rank in ranks:
    reduced_images.append(image_reduced(flower, rank=rank))
fig, ax = plt.subplots(2, 2)
for i, rank in enumerate(ranks):
    image = reconstruct_image(reduced_images[i])
    plot_image(image, ax=ax.flatten()[i], title=r'Rank=%d' % rank)

From the above images we see that increasing the rank improves our low-rank approximation. Recall, that the full rank of the original image is 427. Despite this, even a rank 40 approximation is pretty good - albeit with some artifacts. At rank 100 the image is indistinguishable (at least to me). The memory requirement for a rank-100 approximation is 9.78 MB, significantly smaller than the original 25.02 MB image.

In [ ]:
def memory_of_approximation(r):
    """
    Returns the memory requirement for a rank-r approximation.
    
    Parameters
    ----------
    r: rank of approximation.

    Returns
    -------
    mem: Memory in MB needed to store the approximation.
    """
    # Have r floats, r vectors of size 427, r vectors of size 640 and 3 channels
    mem = 3*((r*427) + (r*640) + r)*32/1024/1024
    return mem

print('Memory of rank 100 approximation %.2f MB ' % (memory_of_approximation(100)))

A more systematic way, than looking by eye, to gauge what rank is needed is to look at the singular values. We can do this by obtaining a full-rank approximation, and then adding up the singular values for each channel and normalising.

In [ ]:
flower_rank_full = image_reduced(flower)

In [ ]:
# Let's look at the singular values
svd_sum = 0
for channel in channels:
    (U, S, VH) = flower_rank_full[channel]
    plt.semilogy(S[::-1]/np.max(S), color=channel)
    svd_sum += S[::-1]
# Normalise svd_sum
svd_sum /= np.max(svd_sum)
plt.semilogy(svd_sum, color='black')
plt.grid(which='both')
plt.xlabel(r'$j$')
plt.ylabel(r'$\sigma_j/\sigma_0$')

In [ ]:
rank_99 = np.argmax(svd_sum < 0.01)
print(rank_99)

We see that the singular values drop off quickly, indicating that a low-rank approximation will be possible. A good rule of thumb is to set the rank such that the singular values have dropped to below 99% of the their maximum value. For the flower this obtained at 65.

### Exercise
Explore low-rank approximations for another image, e.g. `china.jpg`, in the scikit-image library.

## Part 2: POD

So far we've seen that the SVD is able to capture the essence of a complicated dataset by reducing it to a low-rank approximation. Let's now consider the application of the SVD to fluid mechanics datasets. To this end, let's consider the classic example of flow past a cylinder (at Reynolds number 100). The dataset is stored on Hugging Face and can be downloaded with the `download_data` helper function. First, let's examine the data. The data is stored in *.npz* format and contains a number of numpy arrays
1. The velocity in the $x$-direction 'u' : shape (number of snapshots, $x$-resolution, $y$-resolution)
2. The velocity in the $y$-direction 'v' : shape (number of snapshots, $x$-resolution, $y$-resolution)
3. The $x$-coordinates of the 'u' and 'v' data data 'xu' and 'xv', respectively.
4. The $y$-coordinates of the 'u' and 'v' data data 'yu' and 'yv', respectively.
5. An array of times 't' at which the velocity snapshots are sampled.

Also provided is the lift force on the cylinder at every timestep in the simulation (rather than the coarser time-resolution of the snapshots).

6. The lift data 'lift'
7. The times corresponding to the lift_data 'lift_time'

In [ ]:
# Load data
download_data()
data = np.load('data/cylinder_flow_data.npz')

xu = data['xu']
yu = data['yu']
u = data['u']

xv = data['xv']
yv = data['yv']
v = data['v']
t = data['t']
lift  = data['lift']
lift_time  = data['lift_time']

In [ ]:
plt.plot(lift_time[1:], lift[1:])
plt.xlabel(r'$t$')
plt.ylabel(r'Lift')

In a separate file, mass matrices are provided for the cylinder data. As the data is provided on an non-equispaced mesh, the mass matrices allow for a quantity resembling the kinetic energy to be constructed, i.e.
$$
\textit{KE}=\frac{1}{2}\iint u\cdot u + v\cdot v\;\textrm{d}V\approx\frac{1}{2}\left(\mathbf{u}^T\mathbf{W}_u\mathbf{u} + \mathbf{v}^T\mathbf{W}_v\mathbf{v} \right).
$$
 We illustrate their use by constructing a kinetic energy timeseries for the flow data.

In [ ]:
# Load the mass matrices
mass_u = sp.load_npz('data/cylinder_mass_u.npz')
mass_v = sp.load_npz('data/cylinder_mass_v.npz')
number_of_snapshots = u.shape[0]
# Construct the kinetic energy
KE = []
for (u_snapshot, v_snapshot) in zip(u.reshape(number_of_snapshots, -1), v.reshape(number_of_snapshots, -1)):
    KE.append(0.5*np.vdot(u_snapshot, mass_u@u_snapshot) + 0.5*np.vdot(v_snapshot, mass_v@v_snapshot))
KE = np.array(KE)

plt.plot(t, KE)
plt.xlabel(r'$t$')
plt.ylabel(r'Kinetic energy')

Based on the above images, we can see that the flow evolves to a periodic state which starts at around $t=200$. This is the famus Von-Karnan vortex street. For this notebook let's truncate our flow data to this periodic flow (denoted with an LC for limit-cycle).

In [ ]:
t_start = np.argmin(np.abs(t-300))
u_LC = u[t_start:]
v_LC = v[t_start:]
t_LC = t[t_start:]

Using our helper function for plotting the cylinder data, we can now make an animation of the flow.

In [ ]:
%%time
fig, ax = plt.subplots()
# Set up plot
mesh = plot_cylinder_data(xu, yu, u_LC[0], fig_ax=(fig, ax))
def animate(i):
    mesh.set_array(u_LC[i])
    return mesh,
    
num_frames = 120
anim = matplotlib.animation.FuncAnimation(fig, animate, frames=num_frames)
HTML(anim.to_jshtml())

By averaging, we can also construct the mean flow.

In [ ]:
u_bar = np.mean(u_LC, axis=0)
v_bar = np.mean(v_LC, axis=0)

In [ ]:
plot_cylinder_data(xu, yu, u_bar, fig_ax=None, cmap=None)
plot_cylinder_data(xv, yv, v_bar, fig_ax=None, cmap=None)

By examining the flow-field, it is evident that there is a lot of structure. This visual inspection suggests that the flow is low-rank, i.e. it can be approximated well by only a few modes. This low-rank approximation can be achieved by performing a proper orthogonal decomposition. In order to do this, we proceed as follows. Firstly, we separate the unsteady dynamics from the mean-flow by computing
$\mathbf{q}(t, \mathbf{x})=\bar{\mathbf{q}}(\mathbf{x}) + \mathbf{q}'(t, \mathbf{x})$. Note that, the mean-flow does not depend on time. However, the fluctuations do. We now seek a decomposition of the unsteady dynamics
$$
\mathbf{q}' = \sum_{i=0}^{r-1} a_i(t) \boldsymbol{\phi}_i(\mathbf{x}),
$$
where we have assumed a separation between space and time. By further demanding that our spatial modes $\boldsymbol{\phi}_i$ are orthogonal (in some sense we will define later), this decomposition means that the unsteady-dynamics are being represented using a rank-$r$ approximation. Time dependence is entirely through the time-dependent coefficients $a_i$. The question is then, how to choose this basis optimally? Clearly there are a lot of choices for this basis, but ideally we want the smallest basis possible which captures the key features of the flow. This suggests that we would like a basis where $a_i$ decays quickly, meaning that most of the information is contained in a few dominant modes. Proper orthogonal decomposition provides a way to achieve this.

First of all, let's consider how we want our modes to be orthogonal. If they are orthononal under the $L_2$ inner-product, then 
$$
\mathbf{q}'\cdot \mathbf{q}' = \sum_i a_i(t)^2.
$$
In this manner, we see that $a_i$ is analogous to energy. This may be numerically convenient, however, depending on $\mathbf{q}$ this discrete form of energy might not correspond to a physically meaningful one - for instance, if a non-equispaced grid is used. Ideally, we would like to $a_i$ to represent a physical energy and so we really want to do weighted inner product such that
$$
\mathbf{q}'\cdot \mathbf{W}\mathbf{q}' \approx \\\int_\Omega |q|^2\;\textrm{d}V,
$$
where $\mathbf{W}$ is a suitably chosen symmetric positive definite weight matrix (see above in the notebook, for example), and $q$ is the continous quantity that $\mathbf{q}$ discretises. Based on this, we should take $\boldsymbol{\phi}_i$ orthogonal with respect to the weighted inner product with weight $\mathbf{W}$. This means that 
$$
\boldsymbol{\phi}_i\cdot\mathbf{W}\boldsymbol{\phi}_j = \delta_{ij},
$$
and 
$$
\mathbf{q}'\cdot \mathbf{W}\mathbf{q}' = \sum_i a_i(t)^2 \approx \\\int_\Omega |q|^2\;\textrm{d}V,
$$
showing that $a_i(t)$ now gives the amount of physical energy that mode $\phi_i$ contains.

Armed with an appropriate inner product, we are ready to formulate the problem of finding the best basis. To do this we first recognise that $a_i(t)$ is time-dependent. Therefore, we should try and pick each mode to maximise the time averaged energy
$$
\bar{E_i} = \frac{1}{N}\sum_j a_i(t_j)^2,
$$
where $N$ is the number of snapshots. By arranging our flow snapshots in a matrix
$$\mathbf{A}=
\begin{pmatrix}
\mathbf{q}'(t_0) & \mathbf{q}'(t_1) & \cdots & \mathbf{q}'(t_{N-1})
\end{pmatrix},
$$
we can form $\bar{E_i}$ with the following steps
1. **Project the data**
$$
\begin{pmatrix} 
a_i(t_0) & a_i(t_1) & \cdots & a_i(t_{N-1})
\end{pmatrix} = \phi^T_i\mathbf{W}\mathbf{A}
$$
2. **Form the quantity we wish to maximise**
$$ 
\bar{E_i} = \frac{1}{N}(\phi_i^T\mathbf{W}\mathbf{A})\cdot(\phi^T_i\mathbf{W}\mathbf{A}) = \frac{1}{N}\boldsymbol{\phi}^T_i\mathbf{W}\mathbf{A}\mathbf{A}^T\mathbf{W}^T\boldsymbol{\phi}_i
$$

Clearly from step 2 there is a relation between $\bar{E_i}$, our basis $\{\boldsymbol{\phi}_i\}$ and the snapshot matrix. As our modes have unit energy, this maximisation problem is equivalent to maximising
$$ 
\bar{E_i} = \frac{1}{N}\frac{\boldsymbol{\phi}^T_i\mathbf{W}\mathbf{A}\mathbf{A}^T\mathbf{W}^T\boldsymbol{\phi}_i}{\boldsymbol{\phi}^T_i\mathbf{W}\boldsymbol{\phi}_i}.
$$
If we now write $\tilde{\boldsymbol{\phi}}_i = \mathbf{M}\boldsymbol{\phi}_i$ where $\mathbf{M}$ is found such that $\mathbf{W}=\mathbf{M}^T\mathbf{M}$, this problem takes the form of a *Rayleigh quotient*
$$ 
N\bar{E_i} = \frac{\tilde{\boldsymbol{\phi}}^T_i\mathbf{M}\mathbf{A}\mathbf{A}^T\mathbf{M}^T\tilde{\boldsymbol{\phi}}_i}{\tilde{\boldsymbol{\phi}}^T_i\tilde{\boldsymbol{\phi}}_i}.
$$
Doing this allows us to immediately solve the problem. The solution is obtained by finding the eigenvalues and eigenvectors of $\mathbf{R}=\mathbf{M}\mathbf{A}\mathbf{A}^T\mathbf{M}^T$ (which is the covariance matrix of our snapshots!). In other words, the POD decomposition finds the optimal basis by solving
$$
\mathbf{R}\tilde{\boldsymbol{\phi}}_i = \lambda_i\tilde{\boldsymbol{\phi}}_i,
$$
where $\lambda_i=N\bar{E_i}$. As the covariance matrix is a symmetric positive definite matrix, the eigenvalues are real and greater than zero. Hence, we can rank them $\lambda_0>\lambda_1>...>\lambda_{N-1}>0$. POD works really well when there is a fast fall off in eigenvalues, as then the majority of the time-averaged energy can be explained by only a few modal structures.

### Connection with the SVD
The issue with the POD method as explained above is that the covariance matrix $\mathbf{R}\in\mathbb{R}^{M\times M}$ is a huge matrix for most fluid flow snapshots as the number of spatial degrees of freedom $M$ is usually large. To solve this problem, POD is usually performed using the method of snapshots. This method arises by using the fact that the eigenvalues of $\mathbf{R}$ are related to the singular values of the matrices $\mathbf{A}^T\mathbf{M}^T$ and $\mathbf{M}\mathbf{A}$. Therefore, we can instead find the singular value decomposition of $\mathbf{M}\mathbf{A}$, whose left singular vectors are the eigenvectors of $\mathbf{R}$. This is a much easier problem as it can be solved by finding the eigenvectors of the $N\times N$ matrix
$$
\mathbf{Q}=\mathbf{A}^T\mathbf{W}\mathbf{A}.
$$
This is usually a much easier problem as normally the number of snapshots is significantly less than the number of degress of freedom ($N \ll M$). As the eigenvectors $\mathbf{v}_i$ of this matrix are the right singular vectors of $\mathbf{M}\mathbf{A}$, we recover the left singular vectors (the ones we want) in a final step
$$
\phi_i = \frac{1}{\sqrt{\lambda_i}}\mathbf{A}\mathbf{v}_i.
$$
This is the method we now implement.

In [ ]:
def POD(X, weights=None, rank=10):
    """
    Computes the POD using the method of snapshots.
    
    Parameters
    ----------
    X: List of snapshot matrices. Each element is a snapshot matrix for a physical variable.
       Each matrix can be multidimensional, but time must be the last axis.
    weight: List of weight matrices for weighting the snapshots. Each element conatins
       to the weight matrix for the corresponding element of X.
    
    Returns
    -------
    pod_modes: Matrix of pod_modes (mode_index, variable, space).
    eigval: eigenvalues corresponding to the pod_modes.
    pod_mode_amplitudes: Temporal amplitudes corresponding to the pod_modes
    """
    # Create the snapshot matrix
    orig_shapes = []
    var_lengths = []
    A = []
    for var in X:
        # Store the spatial shape
        orig_shapes.append(var.shape[:-1])
        # Store the spatial degrees of freedom
        var_lengths.append(np.prod(orig_shapes[-1]))
        # Flatten and concatenate variables
        if var.ndim != 2:
            # Must flatten spatial dimensions before SVD
            A.append(var.reshape((var_lengths[-1], -1)))
        else:
            A.append(var)
    # Convert to numpy array
    A = np.vstack(A)
    # Form the weight matrix
    if weights==None:
        W = sp.identity(A.shape[0])
    else:
        W = sp.block_diag(weights)
    # Form the covariance matrix
    Q = A.T @ W @ A
    # Perform the eigenvalue decompositions
    eigval, eigvec = sp.linalg.eigs(Q, k=rank)
    # Reconstruct the POD modes
    pod_modes = A @ eigvec @ np.diag(1/np.sqrt(eigval))
    # Make mode_index the first dimension
    pod_modes = pod_modes.T
    # Project to get mode_amplitudes
    pod_mode_amplitudes = pod_modes @ W @ A
    # Restructure the POD modes for output
    pod_modes_split = np.split(pod_modes, np.cumsum(var_lengths)[:-1], axis=1)
    pod_modes = []
    for pod_mode, orig_shape in zip(pod_modes_split, orig_shapes):
        pod_modes.append(pod_mode.reshape((-1,) + orig_shape))
    return eigval, pod_modes, pod_mode_amplitudes

In [ ]:
# Create POD data by moving time to the last axis
POD_data_u = (u_LC - u_bar).transpose(1, 2, 0)
POD_data_v = (v_LC - v_bar).transpose(1, 2, 0)
# Get a rank 20 POD decomposition
eigval, pod_modes, pod_mode_amplitudes = POD([POD_data_u, POD_data_v], rank=20)

In [ ]:
# Plot the normalise eigenvalues
plt.semilogy(eigval/eigval[0], 'x')
plt.xlabel(r'\lamba_i/\lamba_0')
plt.ylabel(r'POD mode i')
plt.xticks(np.arange(0, 20))
plt.grid();

In [ ]:
# Plot the first POD-mode
plot_cylinder_data(xu, yu, pod_modes[0][0].real, fig_ax=None, cmap='bwr')
plot_cylinder_data(xv, yv, pod_modes[1][0].real, fig_ax=None, cmap='bwr')

### Exercise
Plot the higher order POD modes. Can you explain why they seem to come in pairs?

## Part 3: DMD
As we have seen, POD is able to find an orthogonal basis that captures the average energy of the flow with the fewest number of modes. However, for many fluid applications this may not be the modal decomposition we want to perform. The issue is that whilst the POD modes are linearly uncorrelated, they are nonlinearly correlated. Another way to say this, is that multiple physical processes can be present in each POD mode. In the context of cylinder flow, this can be seen by looking at the time varying amplitudes of each POD mode. 

In [ ]:
# Plot the amplitudes of the first two pairs of modes
first_mode_amplitude = pod_mode_amplitudes[0].real
second_mode_amplitude = pod_mode_amplitudes[2].real
plt.plot(t_LC, first_mode_amplitude, label=r'POD mode 0')
plt.plot(t_LC, second_mode_amplitude, label=r'POD mode 2')
plt.xlim([t_LC[0], t_LC[0]+6])
plt.xlabel(r'time')
plt.ylabel(r'Mode amplitude')
plt.legend()

This plot suggests that the POD modes consist of discrete frequencies. However, performing a fast Fourier transform (FFT) reveals that this is not completely true.

In [ ]:
freqs = np.fft.fftshift(np.fft.fftfreq(len(first_mode_amplitude), d=t_LC[1]-t_LC[0]))
mode_0_fft = np.fft.fftshift(np.abs(np.fft.fft(first_mode_amplitude)))
mode_2_fft = np.fft.fftshift(np.abs(np.fft.fft(second_mode_amplitude)))
plt.semilogy(freqs, mode_0_fft, label=r'POD mode 0')
plt.semilogy(freqs, mode_2_fft, label=r'POD mode 2')
plt.ylim([1e1, 1e5])
plt.xlim([0, 1])
plt.xlabel('Frequency (Hz)')
plt.ylabel('FFT amplitude')
plt.legend()

The frequency spectrum shows that the first mode is predominantly the vortex shedding frequency. However, although the second mode contains mostly frequency content at the first harmonic of the fundamental frequency, this mode also contains some power at the fundamental frequency, as well as further superharmonics. This is a direct consequence of requiring the modes to be orthogonal in space. Physically, we expect that the fundamental vortex shedding mode is generated via instability and saturated via nonlinearities. In this way, perhaps a more physically relevant basis would consist of modes each having a discrete frequency. For more complex fluid flows with multiple physical phenomena, separating out each frequency can result in a clearer to understand modal decomposition, with physical processes that POD modes would blend together being more easily distinguished. This modal decomposition can be achieved by using dynamic mode decomposition (DMD), which we now describe.

The DMD algorithm begins with two snapshot matrices $\mathbf{X}$ and $\mathbf{Y}$ where
$$
\mathbf{X}=
\begin{pmatrix}
\mathbf{q}(t_0) & \mathbf{q}(t_1) & \cdots & \mathbf{q}(t_{N-1})
\end{pmatrix},
$$
$$
\mathbf{Y}=
\begin{pmatrix}
\mathbf{q}(t_1) & \mathbf{q}(t_2) & \cdots & \mathbf{q}(t_{N})
\end{pmatrix}.
$$
Note, that the columns of $\mathbf{Y}$ by construction contain the same snapshots as $\mathbf{X}$, except they are one timestep $\Delta t$ ahead. DMD then assumes that we can linearly approximate the snapshots in $\mathbf{Y}$ from those in $\mathbf{X}$ via a matrix $\mathbf{B}$, i.e.
$$
\mathbf{Y}=\mathbf{B}\mathbf{X}.
$$
We then wish to find the eigenvalues and eigenvectors of $\mathbf{B}$, which will give our DMD basis. The difficulty with this, is that the matrix $\mathbf{B}$ is unknown. Therefore, we first need to approximate it from the data.

To this end, we realise that if we knew the pseudo-inverse of $\mathbf{X}$ (denoted $\mathbf{X}^+$), then we could write 
$\mathbf{B}=\mathbf{Y}\mathbf{X}^+$. It turns out that the SVD is a great way to approximate $\mathbf{X}^+$. To see this, lets take the SVD of $\mathbf{X}$ truncated to the first $r$ largest singular values. This gives a rank $r$ reduction of $\mathbf{X}$
$$
\mathbf{X}_r=\mathbf{U}_r\Sigma_r\mathbf{V}_r^T.
$$
By using the fact that $\mathbf{U}_r$ and $\mathbf{V}_r$ are unitary, we then immediately have that
$$
\mathbf{I}=\mathbf{X}_r (\mathbf{V}_r\Sigma_r^{-1}\mathbf{U}_r^T),
$$
showing that $\mathbf{X}_r^+=\mathbf{V}_r\Sigma_r^{-1}\mathbf{U}_r^T$. This shows that we can approximate $\mathbf{B}$ with
$$
\mathbf{B}_r = \mathbf{Y}\mathbf{V}_r\Sigma_r^{-1}\mathbf{U}_r^T.
$$
Similar to our POD discussion, this matrix is $M\times M$ which is too large to be numerically feasible. Therefore, instead of finding the eigenvalue decomposition of this matrix directly, we instead write
$$
\tilde{\mathbf{B}}_r= \mathbf{U}_r^T\mathbf{B}_r\mathbf{U}_r = \mathbf{U}_r^T\mathbf{Y}\mathbf{V}_r\Sigma_r^{-1}.
$$
As $\mathbf{U}_r$ is unitary, the first equivalencey in the above expression shows that our $r\times r$ matrix $\tilde{\mathbf{B}}_r$ shares the same eigenvalues as the larger matrix $\mathbf{B}_r$. Hence, DMD proceeds by finding the eigenvalues $\mu_i$ and eigenvectors $\tilde{v}_i$ of $\tilde{\mathbf{B}}_r$. From this, the eigenvectors of $\mathbf{B}_r$ can be found via $v_i=\mu_i^{-1}\mathbf{X}\mathbf{V}_r\Sigma_r^{-1}\tilde{v}_i$.  This completes the DMD algorithm.


In [ ]:
def DMD(X, rank=10):
    """
    Performs DMD on snapshot data, where time is the last axis
    and snapshots are separated by a constant time interval.

    Parameters
    ----------
    snapshots: List of snapshot matrices. Each element is a snapshot matrix for a physical variable.
       Each matrix can be multidimensional, but time must be the last axis.
    rank: rank of the approximation of B
    
    Returns
    -------
    eigval: eigenvalues corresponding to the DMD_modes.
    DMD_modes: Matrix of DMD_modes (mode-index, variable, space).
    dmd_mode_amplitudes: Array of amplitudes of the dmd modes.
    """
    orig_shapes = []
    var_lengths = []
    A = []
    for var in X:
        orig_shapes.append(var.shape[:-1])
        var_lengths.append(np.prod(orig_shapes[-1]))
        if var.ndim != 2:
            A.append(var.reshape((var_lengths[-1], -1)))
        else:
            A.append(var)
    A = np.vstack(A)
    X = A[:, :-1]
    Y = A[:, 1:]
    U, S, VH = sp.linalg.svds(X, k=rank)
    Abar = U.conj().T @ Y @ VH.conj().T @ np.diag(1/S)
    eigval, eigvec = np.linalg.eig(Abar)
    dmd_modes = Y @ VH.conj().T @ np.diag(1/S) @ eigvec
    dmd_modes = dmd_modes.T
    indices = np.argsort(np.abs(eigval.imag))
    eigval = eigval[indices]
    dmd_modes = dmd_modes[indices]
    dmd_mode_amplitudes = np.linalg.lstsq(dmd_modes.T, A)[0]
    dmd_modes_split = np.split(dmd_modes, np.cumsum(var_lengths)[:-1], axis=1)
    dmd_modes = []
    for dmd_mode, orig_shape in zip(dmd_modes_split, orig_shapes):
        dmd_modes.append(dmd_mode.reshape((-1,) + orig_shape))
    return eigval, dmd_modes, dmd_mode_amplitudes

In [ ]:
mu, dmd_modes, dmd_mode_amplitudes = DMD([POD_data_u, POD_data_v], rank=20)

With the DMD modes now computed, lets compare them to the POD modes. Plotting the eigenvalues of each mode immediately shows a difference. Whereas the POD mode eigenvalues represent how much total energy of the flow they capture, the DMD mode eigenvalues represents the time-dependence of the mode. As we have performed DMD on the limit cycle data, all the modes are neutral (i.e. they lie on the unit circle). This means the modes do not grow or decay. However, they oscillate at the frequency given by the imaginary part of the eigenvalue. Note, that as for a linear equation $\dot{\mathbf{x}}=\mathbf{L}\mathbf{x}$ the DMD modes are the eigenvalues of $\exp(\mathbf{L}\Delta t)$, it is common to instead convert the eigenvalues $\mu_i$ to $\lambda_i=\log(\mu_i)\Delta t$. We display both below.

In [ ]:
fig = plt.figure(figsize=(8, 4))
ax0 = fig.add_subplot(121)
ax1 = fig.add_subplot(122)
ax = [ax0, ax1]
theta = np.linspace(0, 2*np.pi, 100)
ax[0].plot(mu.imag, mu.real, 'x')
ax[0].plot(np.cos(theta), np.sin(theta))
ax[0].set_xlim([-1, 1])
ax[0].set_ylim([-1, 1])
ax[0].set_aspect('equal')
ax[0].set_xlabel(r'Imag($\mu$)')
ax[0].set_ylabel(r'Real($\mu$)')

l = np.log(mu)/(t_LC[1]-t_LC[0])
ax[1].plot(l.imag, l.real, 'x')
xL, xR = ax[1].get_xlim()
ax[1].plot([xL, xR], [0, 0])
ax[1].set_ylim([-0.1, 0.1])
ax[1].set_xlim()
ax[1].set_xlabel(r'Imag($\lambda$)')
ax[1].set_ylabel(r'Real($\lambda$)')
fig.tight_layout()

In [ ]:
# Plot the first DMD-mode
plot_cylinder_data(xu, yu, dmd_modes[0][0].imag, fig_ax=None, cmap='bwr')
plot_cylinder_data(xv, yv, dmd_modes[1][0].real, fig_ax=None, cmap='bwr')

Although in this example the DMD modes and POD modes are very similar, a difference can be seen if we examine the amplitudes of the data projected onto the DMD modes.

In [ ]:
# Plot the amplitudes of the first two pairs of modes
first_mode_amplitude = dmd_mode_amplitudes[0].real
second_mode_amplitude = dmd_mode_amplitudes[2].real
plt.plot(t_LC, first_mode_amplitude, label=r'DMD mode 0')
plt.plot(t_LC, second_mode_amplitude, label=r'DMD mode 2')
plt.xlim([t_LC[0], t_LC[0]+6])
plt.xlabel(r'time')
plt.ylabel(r'Mode amplitude')
plt.legend()

In [ ]:
freqs = np.fft.fftshift(np.fft.fftfreq(len(first_mode_amplitude), d=t_LC[1]-t_LC[0]))
mode_0_fft = np.fft.fftshift(np.abs(np.fft.fft(first_mode_amplitude)))
mode_2_fft = np.fft.fftshift(np.abs(np.fft.fft(second_mode_amplitude)))
plt.semilogy(freqs, mode_0_fft, label=r'DMD mode 0')
plt.semilogy(freqs, mode_2_fft, label=r'DMD mode 2')
plt.ylim([1e0, 1e5])
plt.xlim([0, 1])
plt.xlabel('Frequency (Hz)')
plt.ylabel('FFT amplitude')
plt.legend()

We see that each DMD mode consists of a single frequency. For flows that are strongly dominated by two or more physical phenomena, the ability of DMD to separate modes based on growth rates and frequencies can be hugely beneficial.

Finally, we note that for periodic data, DMD is equivalent to performing an FFT, albeit a rather slow one. The real advantage of DMD arises when applied to more complex fluid data, where DMD can identify growth rates and damping rates.

### Exercise
Perform DMD on the growing part of the cylinder flow data. Can you determine the growth rate of the instability?

## References

   de Silva, Brian M., Kathleen Champion, Markus Quade,
   Jean-Christophe Loiseau, J. Nathan Kutz, and Steven L. Brunton.
   *PySINDy: a Python package for the sparse identification of
   nonlinear dynamics from data.* arXiv preprint arXiv:2004.08424 (2020)
   [arXiv](https://arxiv.org/abs/2004.08424)

   Kaptanoglu, Alan A., Brian M. de Silva, Urban Fasel, Kadierdan Kaheman, Andy J. Goldschmidt
   Jared L. Callaham, Charles B. Delahunt, Zachary G. Nicolaou, Kathleen Champion,
   Jean-Christophe Loiseau, J. Nathan Kutz, and Steven L. Brunton.
   *PySINDy: A comprehensive Python package for robust sparse system identification.*
   arXiv preprint arXiv:2111.08481 (2021).
   [arXiv](https://arxiv.org/abs/2111.08481)

   Brunton, Steven L., Joshua L. Proctor, and J. Nathan Kutz.
   *Discovering governing equations from data by sparse identification
   of nonlinear dynamical systems.* Proceedings of the National
   Academy of Sciences 113.15 (2016): 3932-3937.
   [DOI](http://dx.doi.org/10.1073/pnas.1517384113)

   Champion, K., Zheng, P., Aravkin, A. Y., Brunton, S. L., & Kutz, J. N. (2020).
   *A unified sparse optimization framework to learn parsimonious physics-informed
   models from data.* IEEE Access, 8, 169259-169271.
   [DOI](https://doi.org/10.1109/ACCESS.2020.3023625)

   Brunton, Steven L., Joshua L. Proctor, and J. Nathan Kutz.
   *Sparse identification of nonlinear dynamics with control (SINDYc).*
   IFAC-PapersOnLine 49.18 (2016): 710-715.
   [DOI](https://doi.org/10.1016/j.ifacol.2016.10.249)

   Kaheman, K., Kutz, J. N., & Brunton, S. L. (2020).
   *SINDy-PI: a robust algorithm for parallel implicit sparse identification
   of nonlinear dynamics.* Proceedings of the Royal Society A, 476(2242), 20200279.
   [DOI](https://doi.org/10.1098/rspa.2020.0279)

   Kaptanoglu, A. A., Callaham, J. L., Aravkin, A., Hansen, C. J., & Brunton, S. L. (2021).
   *Promoting global stability in data-driven models of quadratic nonlinear dynamics.*
   Physical Review Fluids, 6(9), 094401.
   [DOI](https://doi.org/10.1103/PhysRevFluids.6.094401)